In [24]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import pickle

In [25]:
# 測試集數據
account_info = pd.read_csv('./test/test_account_info.csv')
customer_info = pd.read_csv('./test/test_customer_info.csv')
account_transactions = pd.read_csv('./test/test_account_transactions.csv')

In [26]:
account_customer_map = account_info[['account_number', 'customer_id']].drop_duplicates()

# 合併客戶信息
merged_base = pd.merge(
    account_customer_map,
    customer_info.drop('region_code', axis=1),
    on='customer_id',
    how='left'
)
print(merged_base)

# 合併帳戶信息
merged_base = pd.merge(
    merged_base,
    account_info.drop('customer_id', axis=1).drop('account_open_date', axis=1),
    on='account_number',
    how='left'
)

print(f"基礎合併數據形狀: {merged_base.shape}")
print(merged_base.head())

     account_number customer_id  aut_amt  age  income_level
0         ACCT20379     ID19286   142508    7          25.0
1          ACCT5965      ID5583  2218985   57         126.0
2         ACCT23936     ID22722     3623   25          25.0
3         ACCT12076     ID11435   173790   70          25.0
4          ACCT5666      ID5291  5807981   58          25.0
...             ...         ...      ...  ...           ...
6453      ACCT19928     ID18872     1167   28          25.0
6454       ACCT2702      ID2566     1987   21          25.0
6455      ACCT10007      ID9442       29   33          25.0
6456      ACCT22626     ID21456      779   24          25.0
6457       ACCT3921      ID3683       71   27          25.0

[6458 rows x 5 columns]
基礎合併數據形狀: (6458, 7)
  account_number customer_id  aut_amt  age  income_level  is_unreachable  \
0      ACCT20379     ID19286   142508    7          25.0               0   
1       ACCT5965      ID5583  2218985   57         126.0               0   
2      

In [27]:
incomeMapping = {
  "15.0": "0-300K",
  "30.0": "300K-610K",
  "45.0": "310K-600K",
  "70.0": "610K-800K",
  "90.0": "810K-1M",
  "125.0": "1.01M-1.5M",
  "225.0": "1.51M-3M",
  "400.0": "3.01M-5M",
  "750.0": "5.01M-10M",
  "1000.0": "Above 10M",
  "25.0": "0-500K",
  "126.0": "510K-2M",
  "350.0": "2.01M-5M",
  'nan': "missing"
}

In [28]:
# 初始化存儲結構
account_data = {}

# 首先填充帳戶和客戶信息
for _, row in merged_base.iterrows():
    account_number = row['account_number']
    row = row.to_dict()
    if row['income_level']:
        row['income_level'] = incomeMapping[str(row['income_level'])]
    else:
        row['income_level'] = "missing"
    # 創建帳戶數據字典
    account_data[account_number] = {
        'account_info': row,  # 包含所有帳戶和客戶信息
        'transaction_feature' : {},
        'transactions': []  # 初始化空交易列表
    }

In [29]:
channelMapping = {
  1: "Check Batch Processing",
  2: "Batch Salary Transfer",
  3: "AID System Securities Secondary Debit",
  4: "Bancs-Link Counter Transaction/PD-NET/VCCS Online Query Transaction",
  5: "Foreign Exchange Online Transaction",
  6: "Fund Online Transaction",
  7: "WBS",
  8: "Batch Account Opening",
  9: "BIZTALK-CR-CARD-TXN Credit Card Online Transaction including Card APP",
  10: "BATCH-TXN General Batch Processing",
  11: "FEDI-TNX FEDI",
  12: "Voice Online Transaction/VCCS Online Query Transaction",
  13: "Trickle Feed Transaction (API Securities Broker Batch Processing)",
  14: "Inward Remittance",
  15: "(Internet Banking",
  16: "Trickle Feed Transaction/API Batch Processing",
  17: "FEP-ATM-TXN (ATM)",
  18: "DIGITAL-BANK-TXN (Digital Banking)",
  19: "STORED-VALUE-PAYMENT (Value Storage Platform)"
}

codeMapping = {
  1: "Cash Deposit",
  2: "Transfer Deposit",
  3: "Transfer Withdrawal",
  4: "Cash Withdrawal",
  5: "Transfer Out Correction",
  6: "Deposit Interest",
  7: "Cash Deposit",
  8: "Check Deposit",
  9: "Check Transfer",
  10: "Transfer Deposit",
  11: "Media Transfer",
  12: "Transfer Deposit (Passbook-less Transfer)",
  13: "Authorized Transfer In",
  14: "Transfer Withdrawal",
  15: "Transfer Withdrawal",
  16: "Transfer Interest Withdrawal",
  17: "Authorized Transfer Out",
  18: "Cash Withdrawal",
  19: "Transfer to Issue Bank Check",
  20: "Media Transfer",
  21: "Cash Deposit Correction",
  22: "Transfer Out Correction",
  23: "Transfer Out Correction",
  24: "CD Deposit (ATM)",
  25: "CD Transfer In (ATM Transfer In)",
  26: "CD Transfer In",
  27: "CD Transfer Out (ATM Interbank Transfer Out)",
  28: "CD Transfer Out",
  29: "CD Withdrawal (ATM Cash Withdrawal)",
  30: "CD Withdrawal (Interbank Withdrawal)",
  31: "Payment Transfer Out",
  32: "Cross-border Purchase",
  33: "Consumption Debit",
  34: "Cross-border Return",
  35: "Payment Transfer Out",
  36: "CD Transfer Reversal",
  37: "CD Correction",
  38: "CD Correction",
  39: "SP Consumption Reversal",
  40: "Payment Transfer Reversal",
  41: "Media Transfer In",
  42: "Transfer Deposit",
  43: "Media Transfer Out",
  44: "Transfer Withdrawal",
  45: "Card Refund",
  46: "Late Fee",
  47: "Card Consumption",
  48: "Supplementary Health Insurance Premium",
  49: "Income Tax",
  50: "Income Tax",
  51: "Overdraft Interest",
  52: "Check Payment",
  53: "Check Cash Withdrawal",
  54: "Check Transfer"
}

In [30]:
# 統計需要處理的交易總數
total_transactions = len(account_transactions)
print(f"開始處理 {total_transactions} 筆交易...")

# 批次處理交易數據以避免內存問題
batch_size = 50000
num_batches = (total_transactions + batch_size - 1) // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, total_transactions)
    
    print(f"處理交易批次 {i+1}/{num_batches} (行 {start_idx} 到 {end_idx})...")
    
    # 獲取當前批次的交易
    batch_transactions = account_transactions.iloc[start_idx:end_idx]
    
    # 遍歷此批次的交易並添加到相應的帳戶
    for _, txn in batch_transactions.iterrows():
        account_number = txn['account_number']
        
        # 確保帳戶存在於我們的數據結構中
        if account_number in account_data:
            # 將交易轉換為字典並添加到帳戶的交易列表
            txn = txn.to_dict()
            txn["transaction_channel"] = channelMapping[txn["transaction_channel"]]
            txn["transaction_code"] = codeMapping[txn["transaction_code"]]
            account_data[account_number]['transactions'].append(txn)
            if len(account_data[account_number]['transactions']) > 70:
                account_data[account_number]['transactions'] = sorted(
                    account_data[account_number]['transactions'],
                    key=lambda x: (x.get('transaction_date', 0), x.get('transaction_hour', 0))
                )[-70:] 

# 4. 分析數據結構和示例
print("\n分析字典式數據結構...")

# 計算總帳戶數
total_accounts = len(account_data)
print(f"總帳戶數: {total_accounts}")

# 計算有交易記錄的帳戶數
accounts_with_transactions = sum(1 for acc in account_data.values() if len(acc['transactions']) > 0)
print(f"有交易記錄的帳戶數: {accounts_with_transactions}")
print(f"無交易記錄的帳戶數: {total_accounts - accounts_with_transactions}")

# 計算每個帳戶的平均交易數
total_txns = sum(len(acc['transactions']) for acc in account_data.values())
avg_txns_per_account = total_txns / accounts_with_transactions if accounts_with_transactions > 0 else 0
print(f"每個帳戶的平均交易數: {avg_txns_per_account:.2f}")

# 找到交易數量最多的帳戶
max_txns_account = max(account_data.items(), key=lambda x: len(x[1]['transactions']), default=(None, {'transactions': []}))
if max_txns_account[0] is not None:
    print(f"交易數量最多的帳戶: {max_txns_account[0]} 有 {len(max_txns_account[1]['transactions'])} 筆交易")

# 5. 示例：顯示一些帳戶的數據
print("\n示例帳戶數據:")
# 獲取一些帳戶號碼作為示例
sample_accounts = list(account_data.keys())[:5]

for i, account in enumerate(sample_accounts):
    acc_data = account_data[account]
    print(f"\n帳戶 {i+1}: {account}")
    print(f"  客戶ID: {acc_data['account_info']['customer_id']}")
    
    # 顯示交易摘要
    txns = acc_data['transactions']
    print(f"  交易數量: {len(txns)}")
    
    # 如果有交易，顯示前3筆
    if txns:
        print("  交易示例:")
        for j, txn in enumerate(txns[:3]):
            print(f"    交易 {j+1}: 日期={txn.get('transaction_date')}, 金額={txn.get('transaction_amount')}, 方向={'入帳' if txn.get('transaction_direction') == 1 else '出帳'}")


開始處理 52122 筆交易...
處理交易批次 1/2 (行 0 到 50000)...
處理交易批次 2/2 (行 50000 到 52122)...

分析字典式數據結構...
總帳戶數: 6458
有交易記錄的帳戶數: 6458
無交易記錄的帳戶數: 0
每個帳戶的平均交易數: 7.80
交易數量最多的帳戶: ACCT31036 有 70 筆交易

示例帳戶數據:

帳戶 1: ACCT20379
  客戶ID: ID19286
  交易數量: 1
  交易示例:
    交易 1: 日期=18299, 金額=81045, 方向=出帳

帳戶 2: ACCT5965
  客戶ID: ID5583
  交易數量: 2
  交易示例:
    交易 1: 日期=18291, 金額=796, 方向=出帳
    交易 2: 日期=18299, 金額=1503, 方向=出帳

帳戶 3: ACCT23936
  客戶ID: ID22722
  交易數量: 1
  交易示例:
    交易 1: 日期=18298, 金額=397, 方向=出帳

帳戶 4: ACCT12076
  客戶ID: ID11435
  交易數量: 5
  交易示例:
    交易 1: 日期=18298, 金額=54553, 方向=出帳
    交易 2: 日期=18298, 金額=274386, 方向=入帳
    交易 3: 日期=18298, 金額=8023, 方向=出帳

帳戶 5: ACCT5666
  客戶ID: ID5291
  交易數量: 1
  交易示例:
    交易 1: 日期=18291, 金額=1470, 方向=出帳


In [31]:
# 方法2: 將結構轉換為JSON格式（更通用，但可能會很大）
# 注意: 這可能會產生非常大的文件，取決於數據量
try:
    with open('account_data_sample.json', 'w') as f:
        sample_data = {k: account_data[k] for k in list(account_data.keys())}
        json.dump(sample_data, f, indent=2)
except Exception as e:
    print(f"保存JSON時出錯: {e}")

# 7. 示例：如何使用這個數據結構進行分析
print("\n示例分析:")

# 示例1: 計算每個帳戶的交易統計資訊
print("示例1: 計算交易統計資訊")
account_stats = {}
sus_many_txn = []
non_many_txn = []

for account, data in account_data.items():
    txns = data['transactions']
    if not txns:
        continue
    
    # 交易金額統計
    amounts = [t.get('transaction_amount', 0) for t in txns]
    
    # 入帳和出帳交易
    incoming = [t for t in txns if t.get('transaction_direction') == 2]
    outgoing = [t for t in txns if t.get('transaction_direction') == 1]

    # 計算交易日期範圍
    transaction_dates = [t.get('transaction_date') for t in txns]
    min_date = min(transaction_dates) if transaction_dates else 0
    max_date = max(transaction_dates) if transaction_dates else 0
    date_range = max_date - min_date + 1 if min_date != max_date else 1  # 至少為1天
    
    # 計算日均交易筆數
    daily_avg_txn = len(txns) / date_range if date_range > 0 else 0
    
    data['transaction_feature'] = {
        'txn_count': len(txns),
        'total_txn_amount' : sum(amounts),
        'avg_txn_amount': np.mean(amounts) if amounts else 0,
        'max_txn_amount': max(amounts) if amounts else 0,
        'incoming_count': len(incoming),
        'outgoing_count': len(outgoing),
        'incoming_ratio': len(incoming) / len(txns) if txns else 0,
        'daily_avg_txn': daily_avg_txn,
        'txn_date_range': date_range
    }



示例分析:
示例1: 計算交易統計資訊


In [32]:
import json
import os
from datetime import datetime

output_dir = "split_accounts/test"

os.makedirs(output_dir, exist_ok=True)

for account_number, data in account_data.items():
    print(account_number)
    test_file = os.path.join(output_dir, f"{account_number}.json")
    with open(test_file, 'w', encoding='utf-8') as f:
        json.dump(account_data[account_number], f, indent=2, ensure_ascii=False)

ACCT20379
ACCT5965
ACCT23936
ACCT12076
ACCT5666
ACCT21090
ACCT24441
ACCT22952
ACCT9636
ACCT9100
ACCT8107
ACCT29972
ACCT23976
ACCT23977
ACCT23973
ACCT9099
ACCT9637
ACCT23974
ACCT3637
ACCT21088
ACCT25737
ACCT10789
ACCT1624
ACCT15459
ACCT6206
ACCT9640
ACCT1435
ACCT22041
ACCT5955
ACCT18777
ACCT9639
ACCT1434
ACCT149
ACCT530
ACCT15431
ACCT909
ACCT18517
ACCT22192
ACCT29973
ACCT18776
ACCT21371
ACCT13666
ACCT6278
ACCT30391
ACCT25736
ACCT30392
ACCT23975
ACCT21091
ACCT24016
ACCT1433
ACCT21089
ACCT23295
ACCT18226
ACCT23972
ACCT9638
ACCT21087
ACCT31083
ACCT23243
ACCT7852
ACCT24784
ACCT23892
ACCT6260
ACCT23891
ACCT3571
ACCT11121
ACCT17875
ACCT28892
ACCT28076
ACCT17246
ACCT16354
ACCT7895
ACCT15509
ACCT3761
ACCT23957
ACCT11734
ACCT29531
ACCT18201
ACCT16764
ACCT29981
ACCT3464
ACCT12072
ACCT1664
ACCT19564
ACCT22770
ACCT16965
ACCT23372
ACCT1993
ACCT12749
ACCT16117
ACCT1420
ACCT10849
ACCT27109
ACCT29176
ACCT11371
ACCT25282
ACCT2872
ACCT23007
ACCT11411
ACCT29932
ACCT10060
ACCT20044
ACCT31175
ACCT22977
ACCT